In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense
from tensorflow.keras.layers import Dropout
from tensorflow.keras.layers import Conv2D
from tensorflow.keras.layers import MaxPooling2D
from tensorflow.keras.layers import Conv3D
from tensorflow.keras.layers import MaxPooling3D
from tensorflow.keras.layers import Flatten
import tensorflow as tf
from skimage.color import rgb2gray
from skimage.transform import rescale
from sklearn.model_selection import KFold
from sklearn.metrics import mean_squared_error
import cv2

from tensorflow.keras.layers import Input, Dense, Flatten, Dropout, Conv2D, MaxPooling2D, Activation, concatenate
from tensorflow.keras.models import Model

# Load Data & Grayscale Data

In [2]:
counter = 0
gray = list()
for i in range(441):
    temp = list()
    for j in range(4):
        
        
        img = cv2.imread(f'{i}.jpg')
        
        #img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype("int32")
        
        if (img.dtype == np.uint8):
            img = img.astype(float) / 256
        img = np.mean(img,axis=-1)
        img = np.expand_dims(img,axis=2)
        temp.append(img)
        counter += 1
        
    temp = np.array(temp)
    gray.append(temp)

gray = np.array(gray)

In [3]:
gray.shape

(441, 4, 360, 640, 1)

In [4]:
target = pd.read_csv("coordinates.csv")
Y = np.array(target.values.tolist())

N = list()
S = list()
E = list()
W = list()

for i in range(441):
    N.append(gray[i][0])
    S.append(gray[i][1])
    E.append(gray[i][2])
    W.append(gray[i][3])

N = np.array(N)
S = np.array(S)
E = np.array(E)
W = np.array(W)

# Loading Test Data

In [5]:
target = pd.read_csv("test data/coordinates.csv")
Y_test = np.array(target.values.tolist())
counter = 0
gray = list()
for i in range(len(Y_test)):
    temp = list()
    for j in range(4):
        
        
        img = cv2.imread(f'test data/{i}.jpg')
        
        #img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype("int32")
        
        if (img.dtype == np.uint8):
            img = img.astype(float) / 256
        img = np.mean(img,axis=-1)
        img = np.expand_dims(img,axis=2)
        temp.append(img)
        counter += 1
        
    temp = np.array(temp)
    gray.append(temp)

gray = np.array(gray)

In [6]:
N_test = list()
S_test = list()
E_test = list()
W_test = list()

for i in range(len(Y_test)):
    N_test.append(gray[i][0])
    S_test.append(gray[i][1])
    E_test.append(gray[i][2])
    W_test.append(gray[i][3])
    
N_test = np.array(N_test)
S_test = np.array(S_test)
E_test = np.array(E_test)
W_test = np.array(W_test)

# Building the model by hand

In [7]:
def create_convolution_layers(input_img):
    model = Conv2D(8, (3,3), activation='relu', input_shape=N.shape[1:], padding='same')(input_img)
    model = MaxPooling2D()(model)
    model = Conv2D(16, (3,3), activation='relu', padding='same')(model)
    model = MaxPooling2D()(model)
    
    return model

In [8]:
North_input = Input(shape=gray.shape[2:])
N_model = create_convolution_layers(North_input)

South_input = Input(shape=gray.shape[2:])
S_model = create_convolution_layers(South_input)

East_input = Input(shape=gray.shape[2:])
E_model = create_convolution_layers(East_input)

West_input = Input(shape=gray.shape[2:])
W_model = create_convolution_layers(West_input)

conv = concatenate([N_model,S_model,E_model,W_model])
conv = Flatten()(conv)

dense = Dense(512)(conv)
dense = Dropout(0.01)(dense)

output = Dense(2)(dense)

In [9]:
model = Model(inputs=[North_input,South_input,East_input,West_input], outputs=[output])

In [10]:
model.compile(optimizer='adam', loss='mse', metrics=['mean_squared_error'])

In [11]:
class myCallback(tf.keras.callbacks.Callback): 
    def on_epoch_end(self, epoch, logs={}): 
        if(logs.get('loss') < 1700):   
            self.model.stop_training = True

callbacks = myCallback()

model.fit([N,S,E,W], Y,
          batch_size=10,
          epochs=20,
          callbacks=[callbacks])

Epoch 1/20
45/45 [==============================] - 148s 3s/step - loss: 22561.0527 - mean_squared_error: 22561.0527
Epoch 2/20
45/45 [==============================] - 147s 3s/step - loss: 3289.5312 - mean_squared_error: 3289.5312
Epoch 3/20
45/45 [==============================] - 149s 3s/step - loss: 2352.0315 - mean_squared_error: 2352.0315
Epoch 4/20
45/45 [==============================] - 145s 3s/step - loss: 1405.2280 - mean_squared_error: 1405.2280


In [12]:
a = model.predict([N_test, S_test,E_test,W_test])

# Test MSE

In [13]:
print(mean_squared_error(a,Y_test))

2200.045129223851


# Validation MSE

In [14]:
target = pd.read_csv("final validation data/coordinates.csv")
Y_test = np.array(target.values.tolist())
counter = 0
gray = list()
for i in range(len(Y_test)):
    temp = list()
    for j in range(4):
        
        
        img = cv2.imread(f'final validation data/{i}.jpg')
        
        #img = cv2.cvtColor(image, cv2.COLOR_BGR2GRAY).astype("int32")
        
        if (img.dtype == np.uint8):
            img = img.astype(float) / 256
        img = np.mean(img,axis=-1)
        img = np.expand_dims(img,axis=2)
        temp.append(img)
        counter += 1
        
    temp = np.array(temp)
    gray.append(temp)

gray = np.array(gray)

# delete noisy
gray = np.delete(gray,[14, 15, 35, 55,],axis=0)
Y_test = np.delete(Y_test,[14, 15, 35, 55],axis=0)

N_test = list()
S_test = list()
E_test = list()
W_test = list()

for i in range(len(Y_test)):
    N_test.append(gray[i][0])
    S_test.append(gray[i][1])
    E_test.append(gray[i][2])
    W_test.append(gray[i][3])
    
N_test = np.array(N_test)
S_test = np.array(S_test)
E_test = np.array(E_test)
W_test = np.array(W_test)

In [15]:
a = model.predict([N_test, S_test,E_test,W_test])

In [16]:
print(mean_squared_error(a,Y_test))

2289.6031789070203
